In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
columns = {0:'datetime',1:'open',2:'high',3:'low',4:'close',5:'volume'}

btc_raw = pd.read_csv("data/raw/btcusd.csv", header=None)
eth_raw = pd.read_csv("data/raw/ethusd60.csv", header=None)

In [3]:
def make_df(raw_df):
    df = raw_df.loc[:,:5]
    df.rename(columns=columns, inplace=True)
    df = df[["datetime", "close"]]
    
    #df["timestamp"] = [pd.Timestamp(dt.datetime.fromtimestamp(date)) for date in df["datetime"]]
    df["timestamp"] = df["datetime"].apply(lambda x: dt.datetime.fromtimestamp(x))
    df = df.drop(columns="datetime")
    df.set_index("timestamp", inplace=True)
    df.index=pd.to_datetime(df.index)
    return df

In [4]:
btc = make_df(btc_raw)
eth = make_df(eth_raw)

In [9]:
btc.tail(20)
btc.rename(columns={"close":"btc"}, inplace=True)
eth.rename(columns={"close":"eth"}, inplace=True)

In [13]:
# print(btc.shape, eth.shape)


(54744, 1)

In [14]:
# crypto_df = pd.concat([btc,eth], axis=1,join="inner")
crypto_df = btc.merge(eth, how="inner", on="timestamp")


In [15]:
crypto_df.head()

,btc,eth
timestamp,,
2015-08-07 09:00:00,277.40328,3.00
2015-08-07 12:00:00,281.95386,3.00
2015-08-08 01:00:00,279.67200,3.00
2015-08-08 04:00:00,283.77999,2.00
2015-08-08 11:00:00,273.38930,1.02


In [39]:
num_ticks = 60
symbols = ["btc","eth"]
returns_df_list = []
for symbol in symbols:
    name = f"{symbol}_24_Return"
    returns_df = pd.DataFrame()
    returns_df[name] = crypto_df[symbol].pct_change(periods=24).shift(-24)
    returns_df_list.append(returns_df)

In [40]:
shifted_df = pd.DataFrame()
for symbol in symbols:
    shifted_df[symbol] = crypto_df[symbol].copy()
    for i in range(1,num_ticks):
        name = f"{symbol}-{i}"
        shifted_df[name] = crypto_df[symbol].shift(i)
shifted_df.shape

(45978, 120)

In [82]:
shifted_df.head()

,btc,btc-1,btc-2,btc-3,btc-4,btc-5,btc-6,btc-7,btc-8,btc-9,...,eth-50,eth-51,eth-52,eth-53,eth-54,eth-55,eth-56,eth-57,eth-58,eth-59
timestamp,,,,,,,,,,,,,,,,,,,,,
2015-08-07 09:00:00,277.40328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-07 12:00:00,281.95386,277.40328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-08 01:00:00,279.67200,281.95386,277.40328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-08 04:00:00,283.77999,279.67200,281.95386,277.40328,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-08 11:00:00,273.38930,283.77999,279.67200,281.95386,277.40328,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
returns_df_list[0].shape

(45978, 1)

In [42]:
from sklearn.linear_model import LinearRegression

In [83]:
btc_concat = pd.concat([shifted_df.pct_change(), returns_df_list[0]], axis=1)
btc_concat.dropna(inplace=True)
btc_concat.shape

eth_concat = pd.concat([shifted_df.pct_change(), returns_df_list[1]], axis=1)
eth_concat.dropna(inplace=True)

In [44]:
X = btc_concat.iloc[:,:120].values
y = btc_concat.iloc[:,-1].values


In [46]:
btc_model = LinearRegression()
btc_model.fit(X,y)

LinearRegression()

In [47]:
predicted_btc = btc_model.predict(X)

In [55]:
predicted_df = pd.DataFrame({"actual": y, "predicted":predicted_btc})
predicted_df.reset_index(inplace=True)
predicted_df.drop(columns=["index"], inplace=True)
predicted_df.to_csv("plot_data.csv")

In [58]:
from sklearn.metrics import r2_score
r2 = r2_score(y,predicted_btc)
print(r2)

0.0037583232960800572


In [65]:
buy_df = pd.DataFrame()
buy_df["signal"] = btc_concat.iloc[:,-1].apply(lambda x: 1 if x > 0.01 else 0)
buy_df.head(20)

,signal
timestamp,
2015-08-15 08:00:00,0
2015-08-15 09:00:00,0
2015-08-15 10:00:00,0
2015-08-15 12:00:00,0
2015-08-15 13:00:00,0
2015-08-15 14:00:00,0
2015-08-15 16:00:00,0
2015-08-15 20:00:00,0
2015-08-15 22:00:00,0


In [66]:
from sklearn.model_selection import train_test_split
X=X
y = buy_df["signal"].values
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y)

In [67]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver="lbfgs")
classifier.fit(X_train, y_train)
classifier.score(X_train, y_train)

0.6328878558977339

In [68]:
predictions = classifier.predict(X_test)
np.sum(predictions)

12

In [69]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[7252,    8],
       [4210,    4]], dtype=int64)

In [70]:
from sklearn.metrics import classification_report
target_names = ["Not Buy", "Buy"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

     Not Buy       0.63      1.00      0.77      7260
         Buy       0.33      0.00      0.00      4214

    accuracy                           0.63     11474
   macro avg       0.48      0.50      0.39     11474
weighted avg       0.52      0.63      0.49     11474



In [71]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500)
rf_model = rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

In [72]:
print(np.sum(y_test), np.sum(rf_predictions))

4214 938


In [73]:
print(classification_report(y_test, rf_predictions, target_names=target_names))

              precision    recall  f1-score   support

     Not Buy       0.65      0.94      0.77      7260
         Buy       0.52      0.12      0.19      4214

    accuracy                           0.64     11474
   macro avg       0.58      0.53      0.48     11474
weighted avg       0.60      0.64      0.55     11474



In [74]:
from sklearn.ensemble import GradientBoostingClassifier

In [77]:
gb_classifier = GradientBoostingClassifier(n_estimators=30,
                                    learning_rate=0.5,
                                    max_features=10,
                                    max_depth=6)
gb_classifier.fit(X_train,y_train


gb_predictions = gb_classifier.predict(X_test)

In [78]:
print(classification_report(y_test, gb_predictions, target_names=target_names))

              precision    recall  f1-score   support

     Not Buy       0.65      0.85      0.74      7260
         Buy       0.47      0.23      0.30      4214

    accuracy                           0.62     11474
   macro avg       0.56      0.54      0.52     11474
weighted avg       0.59      0.62      0.58     11474



In [79]:
print(np.sum(y_test), np.sum(gb_predictions))

4214 2030
